In [ ]:
from datetime import datetime
from pkg_resources import get_distribution, DistributionNotFound
from strsimpy.cosine import Cosine
import pandas as pd
import re
import requests as requests
import sqlite3
import string
import urllib
import yaml
from xml.etree import ElementTree
from tdda import rexpy
import scoped_mapping

In [ ]:
biosample_sqlite_file = "/Users/MAM/Documents/gitrepos/biosample-analysis/target/harmonized_table.db"
# TODO process these as a list?
ncbitaxon_sqlite_file = "/Users/MAM/Documents/gitrepos/semantic-sql/db/ncbitaxon.db"
envo_sqlite_file = "/Users/MAM/Documents/gitrepos/semantic-sql/db/envo.db"
ncbitaxon_cnx = sqlite3.connect(ncbitaxon_sqlite_file)
envo_cnx = sqlite3.connect(envo_sqlite_file)
target_onto_prefix = 'ENVO'
chars_to_whiteout = '._-'
my_query_fields = ''
my_row_req = 3

env_package_overrides = {
    'built environment': 'built',
    'misc environment': 'miscellaneous',
    'missing': 'no environmental package',
    'unknown': 'no environmental package',
    'default': 'no environmental package',
    'unspecified': 'no environmental package',
    'not available': 'no environmental package',
    'not collected': 'no environmental package'
}

In [ ]:
biosample_cnx = sqlite3.connect(biosample_sqlite_file)

In [ ]:
# Sample of the data we're working with
q = """
select
    id,
    env_package,
    package,
    package_name,
    host_taxid,
    taxonomy_id,
    env_broad_scale,
    env_local_scale,
    env_medium
    from biosample b
limit 10
"""
biosample_first_ten = pd.read_sql(q, biosample_cnx)
biosample_first_ten

In [ ]:
# Get the canonical checklist and package terms from NCBI
# Unfortunately it doesn't do a very good job of differentiating 
# checklists (MIMAG, MIMARKS, etc.) 
# from packages (soil, water, etc.)
# what about ba , euk, etc?
package_dictionary = scoped_mapping.get_package_dictionary()
package_dictionary.to_sql('package_dictionary', biosample_cnx, if_exists='replace', index=False)
package_dictionary

In [ ]:
# Do the Biosample checklist/package fields match any of the cannonical values?
# How many Biosample rows are there?
q = """
select count(*) as biosample_row_count
from biosample b
"""
[biosample_row_count, query_duration] = scoped_mapping.timed_query(q, biosample_cnx, print_timing=False)

print(query_duration)
biosample_row_count

In [ ]:
# How many of those rows can be inner-joined with the canonical checklists/packages?
# Specifically, joining biosample.package_name = package_dictionary.DisplayName
# TODO add indexing to docs and or makefile
# create index biosample_package_name_idx on biosample(package_name);
# create index package_dictionary_DisplayName_idx on package_dictionary(DisplayName);
# create index biosample_package_idx on biosample(package);
# create index biosample_p_pn_idx on biosample(package, package_name);
q = """
select
    count(*) as cannonical_package_name_count
from
    biosample b
inner join package_dictionary pd on
    b.package_name = pd.DisplayName
"""
[cannonical_package_name_count, query_duration] = scoped_mapping.timed_query(q, biosample_cnx, print_timing=True)

print(query_duration)

cannonical_package_name_count

In [ ]:
# What do the combinations of package and package_name look like in the Biosample dataset?
q = """
select
    package,
    package_name,
    count(*) as count
from
    biosample b
group by
    package ,
    package_name
order by
    package ,
    package_name
"""
[package_name_combos, query_duration] = scoped_mapping.timed_query(q, biosample_cnx, print_timing=True)

print(query_duration)

package_name_combos

In [ ]:
# What about the Biosample env_package values?
# Are they also a small, highly regular set?
q = """
select
    env_package,
    count(*) as count
from
    biosample b
group by
    env_package
order by
    count(*) desc
"""
[env_package_count, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)

print(query_duration)

env_package_count

In [ ]:
# env_package is going to need some cleanup
# First, get a set of all canonical env_package values
package_dictionary = scoped_mapping.make_tidy_col(package_dictionary, 'EnvPackage', 'eptidy')
package_dictionary =scoped_mapping.make_tidy_col(package_dictionary, 'EnvPackageDisplay', 'epdtidy')
# update in sqlite
package_dictionary.to_sql('package_dictionary', biosample_cnx, if_exists='replace', index=False)
valid_combo = []
valid_combo = scoped_mapping.add_unique_to_list(valid_combo, package_dictionary['eptidy'])
valid_combo = scoped_mapping.add_unique_to_list(valid_combo, package_dictionary['epdtidy'])

valid_combo

In [ ]:
# determine ID patterns
q = """
select
    distinct stanza
    from statements s
where
    predicate = 'rdf:type'
    and "object" = 'owl:Class'
    and stanza = subject"""
# include non-envo IDs that come from envo?
[ids_from_envo, query_duration] = scoped_mapping.timed_query(q, envo_cnx)
print(query_duration)
ids_from_envo = scoped_mapping.add_prefix_col(ids_from_envo, 'stanza', 'prefix')

id_patterns = scoped_mapping.get_multi_term_patterns(ids_from_envo, 'stanza', 'prefix')

env_package_normalized = scoped_mapping.env_package_nomralizastion(env_package_count, 'env_package',
                                                                target_onto_prefix, id_patterns['ENVO'])

env_package_normalized = scoped_mapping.add_overrides(env_package_normalized, 'remaining_tidied', 'ep_override',
                                                   env_package_overrides)

env_package_normalized = scoped_mapping.flag_canonical(env_package_normalized, 'ep_override', 'is_canonical',
                                                    valid_combo)

env_package_normalized.to_sql('env_package_normalized', biosample_cnx, if_exists='replace', index=False)


In [ ]:
# What do the successful normalizations look like?
q = """
select
    env_package,
    count,
    lhs,
    extract,
    ep_override
from
    env_package_normalized
where
    is_canonical = 1
"""
[successful_normalizastions, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)

print(query_duration)

successful_normalizastions

In [ ]:
# Are there any normalization failures?
q = """
select
    env_package,
    count,
    lhs,
    extract,
    ep_override
from
    env_package_normalized
where
    is_canonical = 0
"""
[normalizastion_failures, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)

print(query_duration)

normalizastion_failures

In [ ]:
# utilizing ncbtitaxon for broad subsetting
# specifically, flag the biosamples whose taxon_id indicates they are an unclassified entity
# ignoring the others will throw out samples OF multicellular organisms, like fruit flies
# Add previous notes about what kinds of samples are missed by this bifurcation
# like bacteria.unclassified_bacteria

q = """
select
    distinct s.subject
from
    entailed_edge ee
join statements s on
    ee.subject = s.subject
where
    ee.predicate = 'rdfs:subClassOf'
    and ee.object = 'NCBITaxon:2787823'
    and s.predicate = 'rdfs:label'
"""
[unclassified_taxa, query_duration] = scoped_mapping.timed_query(q, ncbitaxon_cnx)
unclassified_taxa['unclassified'] = True

print(query_duration)

unclassified_taxa

In [ ]:
# SLOW... CHECK INDICES
# Get the taxonomy_id values from the Biosamples
q = """
select
    taxonomy_id biosample_taxid,
    count(*) as count
from
    biosample b
group by
    taxonomy_id
order by
    count(*) desc
"""
[biosample_tax_id_counts, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)
biosample_tax_id_counts['curie'] = 'NCBITaxon:' + biosample_tax_id_counts['biosample_taxid'].astype(str)

print(query_duration)

In [ ]:
# Merge the two taxon id datasets
# I.e. flag the the Biosample records whose taxonomy_id field belongs to a subclass of 'unclassified entries'.
biosample_tax_id_counts = biosample_tax_id_counts.merge(unclassified_taxa, left_on='curie',
                                                        right_on='subject', how='left')
biosample_tax_id_counts.unclassified.fillna(False, inplace=True)

biosample_tax_id_counts

In [ ]:
# should really add labels to all of them
q = """
select
    subject ,
    value
from statements
where
    predicate = 'rdfs:label' and subject = stanza
"""
[all_tax_labels, query_duration] = scoped_mapping.timed_query(q, ncbitaxon_cnx)

biosample_tax_id_counts = biosample_tax_id_counts.merge(all_tax_labels, left_on='curie',
                                                        right_on='subject', how='left')

biosample_tax_id_counts = biosample_tax_id_counts[['curie', 'biosample_taxid', 'count', 'unclassified', 'value']]
biosample_tax_id_counts.columns = ['curie', 'biosample_taxid', 'count', 'unclassified', 'label']

print(query_duration)
biosample_tax_id_counts.to_sql('biobiosample_tax_id_counts', biosample_cnx, if_exists='replace', index=False)

biosample_tax_id_counts

Almost all of the taxa that are common in the biosample collection are either unclassified/metagenomes or easily recognized cellular organisms

exceptions include:
- 32630 = synthetic construct (other entries; other sequences; artificial sequences)
    - 'other entries' would add 16k rows on top of the 1k 'unclassified entities'
    - metagenomes account for 331 of the 'unclassified entities'
    - there are also a small number of uncultured/unclassified microorganisms in the biosample dataset
- 77133 = uncultured bacterium (cellular organisms; Bacteria; environmental samples)
    - 'cellular organisms' would add 2M rows on top of the 1k 'unclassified entities'
    - 'cellular organisms; Bacteria; environmental samples' adds 26k
    
----

In [ ]:
# Get a table of scoped mixs annotations to be mapped to ontology classes.
biosample_col_to_map = 'env_broad_scale'
scoping_col = 'env_package_normalized.ep_override'
scoping_value = 'water'
# In this case, the scoping includes an inner join requirement for 'unclassified entities'

q = 'select ' + biosample_col_to_map + """, count(*) as count
from
    biosample b
join env_package_normalized on
    b.env_package = env_package_normalized.env_package
inner join biobiosample_tax_id_counts stic on
    b.taxonomy_id = stic.biosample_taxid
where """ + scoping_col + " = '" + scoping_value + \
    "' group by " + biosample_col_to_map + """
order by
    count(*) desc"""
[mapping_candidates, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)

mapping_candidates

In [ ]:
# The Biosample format allows for pipe-delimited environmental package lists
# Separate those out into their components
multi_frames = []
for row in mapping_candidates.itertuples(index=True, name='Pandas'):
    split_check = row.env_broad_scale
    if split_check is None:
        split_check = ''
    splitted = pd.Series(split_check.split("|"))
    splitted_count = len(splitted)
    repeated = [split_check] * splitted_count
    repeated = pd.Series(repeated)
    as_frame = pd.DataFrame(dict(repeated=repeated, splitted=splitted)).reset_index()
    multi_frames.append(as_frame)
concat_frame = pd.concat(multi_frames)
concat_frame = concat_frame[['repeated', 'splitted']]
mapping_candidates = mapping_candidates.merge(concat_frame, left_on=biosample_col_to_map,
                                              right_on='repeated', how='left')

mapping_candidates



In [ ]:
# do the spliting and extraction here

# Now try to extract ontology terms that are already present
candidate_series_decomposition = scoped_mapping.decompose_series(mapping_candidates['splitted'], id_patterns[target_onto_prefix])
mapping_candidates = pd.concat([mapping_candidates, candidate_series_decomposition], axis=1)

mapping_candidates

In [ ]:
# And join the extracted IDs with their labels
# start by conencting to the rdftab database 
# from which the terms and label-like annotatiosn will be obtained
ontodb = '/Users/MAM/Documents/gitrepos/semantic-sql/db/' + target_onto_prefix.lower() + '.db'
ontocon = sqlite3.connect(ontodb)

In [ ]:
q = """
select
    subject ,
    value
from
    statements s
where
    predicate = 'rdfs:label'
"""
[onto_labels, query_duration] = scoped_mapping.timed_query(q, ontocon)

onto_labels

In [ ]:
mapping_candidates = mapping_candidates.merge(onto_labels, left_on='extract', right_on='subject', how='left')
mapping_candidates

In [ ]:
# Use cosine string distance to see if the labels match
# I.e. the labels claimed by the Biosample data set and the labels asserted in the ontology
# if they're close enough, consider the assigned ID legit
# how close is close enough?
my_cosine_obj = Cosine(1)
mapping_candidates['value'] = mapping_candidates['value'].fillna('')
mapping_candidates['cosine'] = mapping_candidates.apply(
    lambda my_row: my_cosine_obj.distance(my_row['remaining_tidied'].lower(), my_row['value'].lower()), axis=1)
mapping_candidates

In [ ]:
# Get ready to join in the other direction
# I.e. trying to find ontology term IDs based on perfect label matches. Be careful not to reuse column names.
mapping_candidates.columns = ['env_broad_scale', 'count', 'repeated', 'splitted', 'string', 'extract',
                              'remaining_string', 'remaining_tidied', 'term_id', 'lab_from_id', 'lfi_cosine']
mapping_candidates = mapping_candidates.merge(onto_labels, left_on='remaining_tidied', right_on='value', how='left')
mapping_candidates.columns = ['env_broad_scale', 'count', 'repeated', 'splitted', 'string', 'extract',
                              'remaining_string', 'remaining_tidied', 'term_id', 'lab_from_id',
                              'lfi_cosine', 'term_id_from_lab', 'value']

In [ ]:
mapping_candidates

In [ ]:
# Record a consensus
# If either merging on codes or labels was successful.
# cosines for first pass check on assigned IDs still haven't been filtered?
mapping_candidates['consensus_id'] = mapping_candidates['term_id']
mapping_candidates['consensus_id'][mapping_candidates['consensus_id'].isnull()] = \
    mapping_candidates['term_id_from_lab'][mapping_candidates['consensus_id'].isnull()]
mapping_candidates['consensus_lab'] = mapping_candidates['lab_from_id']
mapping_candidates['consensus_lab'][mapping_candidates['consensus_lab'] == ''] = \
    mapping_candidates['value'][mapping_candidates['consensus_lab'] == '']
# mapping_candidates.to_sql('mapping_scratch', biosample_cnx, if_exists='replace', index=False)


```
<ipython-input-49-3e62557cf6d9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapping_candidates['consensus_id'][mapping_candidates['consensus_id'].isnull()] = \
<ipython-input-49-3e62557cf6d9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame
```

In [ ]:
mapping_candidates

In [ ]:
# For which Biosample annotations were not mappings by merging found?
# It looks like remaining_tidied is retaining too much punctuation
# and loosing useful digits (relative to remaining_string)?
# Should try harder to parse not-quite-right embedded IDs like ...
needs_search = mapping_candidates.remaining_tidied[mapping_candidates.consensus_id.isna()]
needs_search_counts = needs_search.value_counts()

needs_search_counts

In [ ]:
# Use a search engine
# For the mixs annotations that didn't already have cannonical IDs or labels
ebs_raw_list = list(needs_search_counts.index)
ebs_raw_list

In [ ]:
# get whiteout frame and relateds
ebs_wo_frame = scoped_mapping.get_whiteout_frame(ebs_raw_list, replaced_chars=chars_to_whiteout)

ebs_wo_frame

In [ ]:
ebs_wo_list = scoped_mapping.get_wo_list(ebs_wo_frame)
ebs_wo_list

In [ ]:
# slow... turn logging back on to show status?
ebs_search_res = scoped_mapping.search_get_annotations_wrapper(ebs_wo_list, bad_chars=chars_to_whiteout, cat_name=biosample_col_to_map,
                                                ontoprefix=target_onto_prefix.lower(), query_fields='', rr=5)
my_best_acceptable = scoped_mapping.get_best_acceptable(ebs_search_res)

my_best_acceptable

In [ ]:
no_acceptable_mappings = scoped_mapping.get_no_acceptable_mappings(ebs_search_res, my_best_acceptable)

no_acceptable_mappings

- Some broad scales look like place names
- Some get a good hit if 'biome' is added
- how to manually review and then add back in?
- add to biosample SQLite database:
    - no_acceptable_mappings
    - my_best_acceptable
    - ebs_search_results (no acceptable + all acceptable)?
    - mapping_candidates -> mapping_scratch (ID-based and exact-tidied-label-based)